In [1]:
%pylab
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import time,urllib,urllib2,cStringIO,logging,datetime,webbrowser,\
IPython.display,shutil,sys, os
import numpy as np
from mpl_toolkits.basemap import Basemap, cm
from scipy.io import netcdf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from IPython.core.display import Image
from PIL import Image as PILImage

/Users/ChowdaHead/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1069: UserWarning: Bad val "TkAgg or echo backend : Agg" on line #1
	"backend : TkAgg or echo backend : Agg
"
	in file "/Users/ChowdaHead/.matplotlib/matplotlibrc"
	Key backend: Unrecognized backend string "tkagg or echo backend : agg": valid strings are [u'pgf', u'cairo', u'MacOSX', u'CocoaAgg', u'gdk', u'ps', u'GTKAgg', u'nbAgg', u'GTK', u'Qt5Agg', u'template', u'emf', u'GTK3Cairo', u'GTK3Agg', u'WX', u'Qt4Agg', u'TkAgg', u'agg', u'svg', u'GTKCairo', u'WXAgg', u'WebAgg', u'pdf']
  (val, error_details, msg))


Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import tarfile
 
def untar(fname):
    if (fname.endswith("tar.gz")):
        tar = tarfile.open(fname)
        tar.extractall()
        tar.close()
        print "Extracted in Current Directory"
    else:
        print "Not a tar.gz file: '%s '" % sys.argv[0]
        

In [3]:
def Rain24hrTotal(year,month,day,Daypath,mypath):
    untar("/Users/ChowdaHead/Desktop/nws_precip_"+Daypath+"_nc.tar.gz")
    
    if month < 10:
        month = "0"+str(month)
    if day < 10:
        day = "0"+str(day)
        
    # plot rainfall from NWS using special precipitation
# colormap used by the NWS, and included in basemap.
  
    nc = netcdf.netcdf_file("/Users/ChowdaHead/Desktop/Weather_Blog/"+\
    "NWS_24hr_Precip_Archive"+\
    "/Conus/nws_precip_conus_"+Daypath+".nc")

    PrecipFilename = str(Year)+"_"+str(Month)+"_"+str(Day)+"_nws_precip_conus.png"
# data from http://water.weather.gov/precip/
    prcpvar = nc.variables['amountofprecip']
    data = 0.01*prcpvar[:]
    latcorners = nc.variables['lat'][:]
    loncorners = -nc.variables['lon'][:]
    lon_0 = -nc.variables['true_lon'].getValue()
    lat_0 = nc.variables['true_lat'].getValue()
    nc.close()
# create figure and axes instances
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
# create polar stereographic Basemap instance.
    m = Basemap(projection='stere',lon_0=lon_0,lat_0=90.,lat_ts=lat_0,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[2],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[2],\
            rsphere=6371200.,resolution='l',area_thresh=10000)
# draw coastlines, state and country boundaries, edge of map.
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
# draw parallels.
    parallels = np.arange(0.,90,10.)
    m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
    meridians = np.arange(180.,360.,10.)
    m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
    ny = data.shape[0]; nx = data.shape[1]
    # get lat/lons of ny by nx evenly space grid.
    lons, lats = m.makegrid(nx, ny) 
    x, y = m(lons, lats) # compute map proj coordinates.
# draw filled contours.
    clevs = [0,1,2.5,5,7.5,10,15,20,30,40,50,70,100,150,\
             200,250,300,400,500,600,750]
    cs = m.contourf(x,y,data,clevs,cmap=cm.s3pcpn)
# add colorbar.
    cbar = m.colorbar(cs,location='bottom',pad="5%")
    cbar.set_label('mm')
# add title
    plt.title(prcpvar.long_name+' for period ending '+\
              prcpvar.dateofdata)
    plt.savefig(mypath+PrecipFilename)
    plt.savefig(mypath+PrecipFilename)
    plt.axis('off')
    #plt.show()
    plt.close()
    
    return PrecipFilename

In [4]:
print "mypath should be: /Users/ChowdaHead/Desktop/Weather_Blog/NWS_24Hr_Rainfall/?????"

mypath should be: /Users/ChowdaHead/Desktop/Weather_Blog/NWS_24Hr_Rainfall/?????


In [5]:
now = datetime.datetime.now()
Time = raw_input("(C)urrent time or (a)rchive? ")
if Time == "C": 
    Day = now.day
    Year = now.year
    Month = now.month
    
if Time == "a":
    Year = input("Year: ")
    Month = input("Month: ")
    Day = input("Day: ")

if Month < 10:
    Month = "0"+str(Month)
if Day < 10:
    Day = "0"+str(Day)

mypath = raw_input("Where would you like the file? \
(For now you need to provide the full path file)")    
#path = "/Users/ChowdaHead/Desktop/Weather_Blog/UWyo_Soundings/"
#mypath = path+str(Year)+"_"+str(Month)+"_"+str(int(Day))+"_"+str(Hour)+"Z"
Daypath = str(Year)+str(Month)+str(Day)

if not os.path.isdir(mypath):
    os.makedirs(mypath)
os.chdir(mypath)

os.chdir("/Users/ChowdaHead/Desktop/Weather_Blog/NWS_24hr_Precip_Archive/Conus/")
tits = webbrowser.open("http://water.weather.gov/precip/download.php")
fatty = raw_input("Downloaded? ")
if fatty == "y":
    Rain24hrTotal(Year,Month,Day,Daypath,mypath)
    shutil.move("/Users/ChowdaHead/Desktop/nws_precip_"+\
    str(Year)+str(Month)+str(Day)+"_nc.tar.gz"\
    , "/Users/ChowdaHead/.Trash/nws_precip_"+\
    str(Year)+str(Month)+str(Day)+"_nc.tar.gz")

(C)urrent time or (a)rchive? a
Year: 2017
Month: 3
Day: 22
Where would you like the file? (For now you need to provide the full path file)/Users/ChowdaHead/Desktop/
Downloaded? y
Extracted in Current Directory


/Users/ChowdaHead/anaconda/lib/python2.7/site-packages/scipy/io/netcdf.py:297: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)
